In [1]:
import numpy as np

centre = (np.float64(0.4511630102430555), np.float64(0.38399840269097224))

start_image_range = 3.0
target_image_range = 1.5798611111284e-06
image_count = 200
video_fps = 15

image_ranges = np.geomspace(start_image_range, target_image_range, image_count)

pixels = 1000

def create_input(centre, image_range, pixels):
    xmin = np.float64(centre[0] - (image_range / 2))
    xmax = np.float64(centre[0] + (image_range / 2))
    ymin = np.float64(centre[1] - (image_range / 2))
    ymax = np.float64(centre[1] + (image_range / 2))

    # Creates 2 lists containing regularly spaced values, number of points = pixel count
    r1 = np.linspace(xmin, xmax, pixels, dtype=np.float64)
    r2 = np.linspace(ymin, ymax, pixels, dtype=np.float64)

    complex_array = r1 + r2[:,None]*1j # ,None creates a new axis in the matrix
    return np.ravel(complex_array)

In [2]:
step_count = 20

r_channel = np.array([0,225,225,210,210,0])
g_channel = np.array([0,30,127,210,210,0])
b_channel = np.array([0,30,30,30,210,0])

def interp_channel(channel, step_count):
    """Interpolates between sequential items by number of steps, count == step_count * items in list"""
    return np.concatenate([np.linspace(start, end, step_count, endpoint=False, dtype=np.uint8) for start, end in zip(channel, channel[1:])])

r_channel_interp = interp_channel(r_channel, step_count)
g_channel_interp = interp_channel(g_channel, step_count)
b_channel_interp = interp_channel(b_channel, step_count)

colour_count = np.uint32(len(r_channel_interp))

In [3]:
import sys
#!conda install --yes --prefix {sys.prefix} -c conda-forge pyopencl
#!conda install --yes --prefix {sys.prefix} -c conda-forge progressbar2
#!conda install --yes --prefix {sys.prefix} -c conda-forge imageio

In [4]:
algorithm = """
            #pragma OPENCL EXTENSION cl_khr_fp64 : enable
            __kernel void mandelbrot(
                __global double2 *input, 
                __global uchar4 *output, 
                uint const maxiter, 
                __constant uchar *r_channel, 
                __constant uchar *g_channel, 
                __constant uchar *b_channel, 
                uint colourCount)
            {
                int gid = get_global_id(0);
                double tempReal, real = 0;
                double imag = 0;
                output[gid] = 0;
                output[gid].w = 255;
                for(uint curiter = 0; curiter < maxiter; curiter++) 
                {
                    tempReal = real*real - imag*imag + input[gid].x;
                    imag = 2 * real*imag + input[gid].y;
                    real = tempReal;
                    if (real*real + imag*imag > 4.0f)
                    {
                        uint idx = curiter % colourCount;
                        output[gid].x = r_channel[idx];
                        output[gid].y = g_channel[idx];
                        output[gid].z = b_channel[idx];
                        break;
                    }
                }
            }
            """
import pyopencl as cl
import pyopencl.array as cl_array

ctx = cl.create_some_context(interactive=False)
 
kernel = cl.Program(ctx, algorithm).build()

In [ ]:
import IPython
from PIL import Image
import progressbar

queue = cl.CommandQueue(ctx)

mf = cl.mem_flags

iterations = np.uint32(1500)

r_buffer = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=r_channel_interp)
g_buffer = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=g_channel_interp)
b_buffer = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_channel_interp)

images = []

for i, image_range in progressbar.progressbar(enumerate(image_ranges)):
    complex_array = create_input(centre, image_range, pixels)
    input_buffer = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=complex_array)
    cl_output = np.zeros((complex_array.shape[0], 4), dtype=np.uint8)
    output_buffer = cl.Buffer(ctx, mf.WRITE_ONLY, cl_output.nbytes)
    
    kernel.mandelbrot(queue, cl_output.shape, None, input_buffer, output_buffer, iterations, r_buffer, g_buffer, b_buffer, colour_count)
    
    cl.enqueue_copy(queue, cl_output, output_buffer).wait()
    images.append(cl_output.reshape((int(pixels), int(pixels), 4)))
    

In [ ]:
import os
import imageio

writer = imageio.get_writer('video.mp4', fps=video_fps)

for im in images:
    writer.append_data(im)
for _ in range(video_fps * 2):
    writer.append_data(images[-1])
writer.close()

In [ ]:
from IPython.display import Video
Video('video.mp4')